In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv("globalterrorismdb_0718dist.csv",encoding = "ISO-8859-1",engine="python")
df.country.isnull().sum()

0

In [3]:
sampled_df = df.groupby((df['iyear']), group_keys=False).apply(lambda x: x.sample(n=min(len(x), 500)))

In [4]:
sampled_df.dropna(subset='longitude',inplace=True)
sampled_df.dropna(subset='latitude',inplace=True)
sampled_df.dropna(subset='country_txt',inplace=True)

In [5]:
df.columns

Index(['eventid', 'iyear', 'imonth', 'iday', 'approxdate', 'extended',
       'resolution', 'country', 'country_txt', 'region',
       ...
       'addnotes', 'scite1', 'scite2', 'scite3', 'dbsource', 'INT_LOG',
       'INT_IDEO', 'INT_MISC', 'INT_ANY', 'related'],
      dtype='object', length=135)

In [30]:
df_new=pd.DataFrame()
df_new['country']=sampled_df.country_txt
df_new['latitude']=sampled_df.latitude
df_new['longitude']=sampled_df.longitude
df_new['year']=sampled_df.iyear
df_new['targ_type']=sampled_df.targtype1_txt

df_new['wep_typ']=sampled_df.weaptype1_txt
df_new['region']=sampled_df.region_txt
df_new['attack_type'] = sampled_df.attacktype1_txt
#df_new['gang_name']=sampled_df.gname

In [31]:
top10 = sampled_df.gname.value_counts().nlargest(20).index.tolist()

# create boolean mask for the rows where col1 value is in top10
mask = sampled_df.gname.isin(top10)

# filter the rows in the dataframe using the boolean mask
df_new['topgang'] = sampled_df.gname[mask]

In [32]:
df_new['topgang'].replace('Unknown', np.nan, inplace=True)

In [33]:
cf_new=df_new

In [34]:
cf_new.dropna(inplace=True)

In [35]:
cf_new

,country,latitude,longitude,year,targ_type,wep_typ,region,attack_type,topgang
82,United States,40.697132,-73.931351,1970,Military,Incendiary,North America,Facility/Infrastructure Attack,Left-Wing Militants
279,United States,42.331685,-83.047924,1970,Government (General),Incendiary,North America,Facility/Infrastructure Attack,Left-Wing Militants
641,United States,37.547030,-122.314834,1970,Government (General),Explosives,North America,Bombing/Explosion,Left-Wing Militants
449,United States,43.064388,-87.966202,1970,Business,Explosives,North America,Bombing/Explosion,Left-Wing Militants
104,United States,39.758968,-104.876305,1970,Private Citizens & Property,Explosives,North America,Bombing/Explosion,Left-Wing Militants
...,...,...,...,...,...,...,...,...,...
176190,Philippines,14.137428,122.411233,2017,Military,Explosives,Southeast Asia,Bombing/Explosion,New People's Army (NPA)
174578,Afghanistan,31.002735,66.405617,2017,Private Citizens & Property,Explosives,South Asia,Assassination,Taliban
170893,Somalia,2.059819,45.326115,2017,Government (Diplomatic),Explosives,Sub-Saharan Africa,Bombing/Explosion,Al-Shabaab
180319,Afghanistan,34.803709,70.098049,2017,Unknown,Explosives,South Asia,Bombing/Explosion,Taliban


In [36]:
x_axis_ticks = cf_new['year'].unique().tolist()
x_ticks = x_axis_ticks[::2]
x_ticks

[1970,
 1972,
 1974,
 1976,
 1978,
 1980,
 1982,
 1984,
 1986,
 1988,
 1990,
 1992,
 1995,
 1997,
 1999,
 2001,
 2003,
 2005,
 2007,
 2009,
 2011,
 2013,
 2015,
 2017]

In [39]:
import altair as alt
alt.data_transformers.disable_max_rows()
brush = alt.selection(type="interval")
brush2 = alt.selection_interval()

#dropdown options for dropdown Filter
options = cf_new['targ_type'].drop_duplicates().tolist()
new = []
new.append(None)
dropdown_options = new+options
dropdown = alt.binding_select(
    options=dropdown_options,
    name='Target Types',
    labels = ["ALL"]+dropdown_options[1:]
)

brush_dropdown = alt.selection_single(
    fields=['targ_type'],
    #value=[{'targtype1_txt': dropdown_options[0]}],
    bind=dropdown
)
#Radio option for radio filters
r_op = cf_new['wep_typ'].drop_duplicates().tolist()
radio_options = new+r_op
radio = alt.binding_radio(options=radio_options, name="Weapon Types",labels = ["ALL"]+radio_options[1:])
brush_radio = alt.selection_point(fields=['wep_typ'], bind=radio)
selection2 = alt.selection_multi(fields=['region'], bind='legend')
#radio filter


country_selection = alt.selection_multi(fields=['country'])
#brush = alt.selection_single(fields=['year'],type='interval')
x1 = alt.selection_single(fields=['topgang'])


pointx = alt.Chart(cf_new).mark_bar().encode(
    x=alt.X('year:O',axis = alt.Axis(values =x_ticks )),
    y='count(topgang)',
    tooltip=['region:N','attack_type:N'],
    color=alt.condition(brush,'topgang:N',alt.value('lightgray'), 
                        legend = alt.Legend(title='Top Terrorist Gang',orient='right'),
                       scale = alt.Scale(scheme = 'tableau10'))
).properties(height=300,width=400).add_selection(brush2).add_selection(brush_dropdown).transform_filter(brush_dropdown).add_selection(brush_radio).transform_filter(brush_radio)

base1=alt.Chart(cf_new).mark_bar().encode(
    x='count()',
    y="topgang",
    color=alt.condition(brush2,'year:Q',alt.value('lightgray'),
                       legend = alt.Legend(title='Gang Activity',orient='left'),
                       scale = alt.Scale(scheme = 'reds')),
    tooltip=['region:N','attack_type:N']
).properties(height=300,width=200).add_selection(brush).add_selection(brush_dropdown).transform_filter(brush_dropdown).add_selection(brush_radio).transform_filter(brush_radio)



y=base1.properties(title="Top Terrorist Gangs") | pointx.properties(title="Top Terrorist Gang Attacks By Year")
y.save("viz2.html")
y

alt.HConcatChart(...)

In [13]:
new = cf_new.groupby('topgang').size().reset_index(name='count')

In [43]:
new['count'].mean()

292.10526315789474